### Import packages

In [1]:
from xml.dom import minidom
from sys import stdin
from urllib import request
from subprocess import call
import numpy as np
import pandas as pd
import itertools
from tabulate import tabulate
import pyart
from sphere import RegionCoverer, Cell, LatLng, LatLngRect, CellId
from datetime import datetime, timedelta
import time


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



### initialize user-defined variables

In [2]:
"""
date = "2020/01/01"
site = "KCRP"
bucketURL = "http://noaa-nexrad-level2.s3.amazonaws.com"
dirListURL = bucketURL+ "/?prefix=" + date + "/" + site
"""
print("")

### list available files

In [3]:
"""
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

print ("listing files from %s" % dirListURL)

xmldoc = minidom.parse(request.urlopen(dirListURL))
itemlist = xmldoc.getElementsByTagName('Key')
print (len(itemlist) , "keys found...")

for x in itemlist:
    file = getText(x.childNodes)
"""
print("")

### read NEXRAD radar file -- having issue reading from http, temporarily reading from local directory

In [4]:
#https://s3.amazonaws.com/noaa-nexrad-level2/index.html#2020/01/01/KCRP/
#https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/KCRP20200101_000431_V06

#radar = pyart.io.read_nexrad_archive('https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/' + 'KCRP20200101_000431_V06')
#radar = pyart.io.read('https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/' + 'KCRP20200101_000431_V06')
radar = pyart.io.read('data/NEXRAD/KCRP20200101_000431_V06')

### show radar info

In [5]:
radar.info()

altitude:
	data: <ndarray of type: float64 and shape: (1,)>
	long_name: Altitude
	standard_name: Altitude
	units: meters
	positive: up
altitude_agl: None
antenna_transition: None
azimuth:
	data: <ndarray of type: float64 and shape: (6480,)>
	units: degrees
	standard_name: beam_azimuth_angle
	long_name: azimuth_angle_from_true_north
	axis: radial_azimuth_coordinate
	comment: Azimuth of antenna relative to true north
elevation:
	data: <ndarray of type: float32 and shape: (6480,)>
	units: degrees
	standard_name: beam_elevation_angle
	long_name: elevation_angle_from_horizontal_plane
	axis: radial_elevation_coordinate
	comment: Elevation of antenna relative to the horizontal plane
fields:
	reflectivity:
		data: <ndarray of type: float32 and shape: (6480, 1832)>
		units: dBZ
		standard_name: equivalent_reflectivity_factor
		long_name: Reflectivity
		valid_max: 94.5
		valid_min: -32.0
		coordinates: elevation azimuth range
		_FillValue: -9999.0
	differential_reflectivity:
		data: <ndarray of 

### write station info

In [6]:
station_info = ['StationName', 'Product', 'Pattern', 'Latitude', 'Longitude', 'Altitude', 'StartTime']
station_row = []

### confirm station location
### process start datetime

In [7]:
station_name = radar.metadata['instrument_name']
product = radar.metadata['original_container']
pattern = radar.metadata['vcp_pattern']

latitude0 = radar.latitude['data'][0]
longitude0 = radar.longitude['data'][0]
altitude0 = radar.altitude['data'][0]

volume_start = datetime.strptime(radar.time['units'][14:34], '%Y-%m-%dT%H:%M:%SZ')

station_row.extend([station_name, product, pattern, latitude0, longitude0, altitude0, volume_start] )

print(station_info)
print(station_row)

['StationName', 'Product', 'Pattern', 'Latitude', 'Longitude', 'Altitude', 'StartTime']
['KCRP', 'NEXRAD Level II', 35, 27.78401756286621, -97.5112533569336, 43.0, datetime.datetime(2020, 1, 1, 0, 4, 31)]


In [8]:
print(radar.sweep_mode)

{'units': 'unitless', 'standard_name': 'sweep_mode', 'long_name': 'Sweep mode', 'comment': 'Options are: "sector", "coplane", "rhi", "vertical_pointing", "idle", "azimuth_surveillance", "elevation_surveillance", "sunscan", "pointing", "manual_ppi", "manual_rhi"', 'data': array([b'azimuth_surveillance', b'azimuth_surveillance',
       b'azimuth_surveillance', b'azimuth_surveillance',
       b'azimuth_surveillance', b'azimuth_surveillance',
       b'azimuth_surveillance', b'azimuth_surveillance',
       b'azimuth_surveillance', b'azimuth_surveillance',
       b'azimuth_surveillance', b'azimuth_surveillance'], dtype='|S20')}


### confirm number of rays and gates
# Ray attributes: azimuth, elevation, time
# Gate attributes: range
# Ray x Gate attributes: lat, lon, alt

In [9]:
print(radar.nrays)
print(radar.ngates)

6480
1832


### unfold lists of lists

In [10]:
merged_lat = list(itertools.chain.from_iterable(radar.gate_latitude['data']))
merged_lon = list(itertools.chain.from_iterable(radar.gate_longitude['data']))
merged_alt = list(itertools.chain.from_iterable(radar.gate_altitude['data']))
merged_refl = list(itertools.chain.from_iterable(radar.fields['reflectivity']['data']))
merged_velo = list(itertools.chain.from_iterable(radar.fields['velocity']['data']))

### expand time field

In [11]:
time_x1 = [volume_start + timedelta(seconds=s) for s in radar.time['data']]
time_xgates = [val for val in time_x1 for _ in range(radar.ngates)]

In [12]:
volume_start_x1 = [volume_start + timedelta(seconds=0)]

stations = pd.DataFrame(
    {'StationName': station_name, 
     'Product': product, 
     'Pattern': pattern, 
     'Latitude': latitude0,
     'Longitude': longitude0,
     'Altitude': altitude0,
     'StartTime': volume_start_x1     
    })


### visual checks -- make into real tests

In [13]:
nsamples = radar.nrays * radar.ngates
ndata_rays = len(radar.gate_latitude['data'])
ndata_gates = len(radar.gate_latitude['data'][0])
ntime_rays = len(radar.time['data'])
ntime_samples = len(time_xgates)
ndata_samples = len(merged_refl)


if radar.nrays == ntime_rays:
    print('time match ' + str(radar.nrays))
else:
    print('time mismatch')
    
if radar.nrays == ndata_rays:
    print('data match ' + str(radar.nrays))
else:
    print('data mismatch')
    
if radar.ngates == ndata_gates:
    print('data match ' + str(radar.ngates))
else:
    print('data mismatch')    
    
if nsamples == ntime_samples:
    print('time match ' + str(nsamples))
else:
    print('time mismatch')
    
if nsamples == ndata_samples:
    print('data match ' + str(nsamples))
else:
    print('data mismatch')

time match 6480
data match 6480
data match 1832
time match 11871360
data match 11871360


### combine data into dataframe

In [14]:
samples = pd.DataFrame(
    {'GateLat': merged_lat,
     'GateLon': merged_lon,
     'GateAlt': merged_alt,
     'GateTime': time_xgates,
     'Reflectivity': merged_refl,
     'Velocity': merged_velo
    })
print(len(samples))
print(samples[0:5000000:100000])

11871360
           GateLat     GateLon  GateAlt                GateTime Reflectivity  \
0        27.802363  -97.505206     61.0 2020-01-01 00:04:31.183           -8   
100000   29.539148  -95.599637   6782.0 2020-01-01 00:04:36.567           --   
200000   28.019360  -96.740719   1081.0 2020-01-01 00:04:42.062           --   
300000   27.318263  -94.022561  10107.0 2020-01-01 00:04:47.449           --   
400000   26.957580  -96.208735   2969.0 2020-01-01 00:04:52.942           --   
500000   24.383280  -95.555656  14211.0 2020-01-01 00:04:58.332           --   
600000   25.661655  -97.501097   5383.0 2020-01-01 00:05:03.821           --   
700000   27.415115  -97.725206    542.0 2020-01-01 00:05:09.314          6.5   
800000   26.111492 -100.062358   8376.0 2020-01-01 00:05:14.701           --   
900000   27.618271  -98.757781   1997.0 2020-01-01 00:05:20.190           --   
1000000  28.864024 -101.320374  12317.0 2020-01-01 00:05:25.583           --   
1100000  29.089156  -98.948946 

In [15]:
samples[samples.Reflectivity.notnull()]

,GateLat,GateLon,GateAlt,GateTime,Reflectivity,Velocity
0,27.802363,-97.505206,61.0,2020-01-01 00:04:31.183,-8,--
1,27.804522,-97.504495,63.0,2020-01-01 00:04:31.183,-12,--
2,27.806680,-97.503783,65.0,2020-01-01 00:04:31.183,-3,--
3,27.808838,-97.503072,67.0,2020-01-01 00:04:31.183,-3,--
4,27.810997,-97.502360,69.0,2020-01-01 00:04:31.183,-3,--
...,...,...,...,...,...,...
11871355,24.528336,-100.237553,63477.0,2020-01-01 00:11:18.914,--,--
11871356,24.526563,-100.238986,63519.0,2020-01-01 00:11:18.914,--,--
11871357,24.524790,-100.240420,63560.0,2020-01-01 00:11:18.914,--,--
11871358,24.523016,-100.241854,63601.0,2020-01-01 00:11:18.914,--,--


In [31]:
#samples[samples['Reflectivity'] == 'masked']

,GateLat,GateLon,GateAlt,GateTime,Reflectivity,Velocity


In [32]:
samples1 = samples[(samples['Reflectivity'] >= -32) & (samples['Reflectivity'] <= 94.5)]
samples1

,GateLat,GateLon,GateAlt,GateTime,Reflectivity,Velocity
0,27.802363,-97.505206,61.0,2020-01-01 00:04:31.183,-8,--
1,27.804522,-97.504495,63.0,2020-01-01 00:04:31.183,-12,--
2,27.806680,-97.503783,65.0,2020-01-01 00:04:31.183,-3,--
3,27.808838,-97.503072,67.0,2020-01-01 00:04:31.183,-3,--
4,27.810997,-97.502360,69.0,2020-01-01 00:04:31.183,-3,--
...,...,...,...,...,...,...
11869653,27.547467,-97.716142,3837.0,2020-01-01 00:11:18.914,-12.5,--
11869693,27.476573,-97.777324,4999.0,2020-01-01 00:11:18.914,-9.5,-1
11869694,27.474801,-97.778852,5028.0,2020-01-01 00:11:18.914,-3,-0.5
11869696,27.471256,-97.781908,5086.0,2020-01-01 00:11:18.914,-7,--


In [ ]:
samples2 = samples[ ( (samples['Reflectivity'] >= -32) & (samples['Reflectivity'] <= 94.5) ) | (samples['Velocity'] >= -95) & (samples['Velocity'] <= 95) ) ]
samples2

### get cellid from lat /lon = 471sec
# get level 10 parent from cellid = 77sec
# get centroid of cellid = 413sec

In [16]:
"""
s2level = 10

start0 = time.time()
samples['S2LL'] = [LatLng.from_degrees(x, y) for x, y in zip(samples['GateLat'], samples['GateLon'])]
end0 = time.time()
print(end0 - start0)
#57s

start1 = time.time()
samples['S2CellID'] = [CellId().from_lat_lng(xy) for xy in samples['S2LL']]
end1 = time.time()
print(end1 - start1)
#426s

"""






"\ns2level = 10\n\nstart0 = time.time()\nsamples['S2LL'] = [LatLng.from_degrees(x, y) for x, y in zip(samples['GateLat'], samples['GateLon'])]\nend0 = time.time()\nprint(end0 - start0)\n#57s\n\nstart1 = time.time()\nsamples['S2CellID'] = [CellId().from_lat_lng(xy) for xy in samples['S2LL']]\nend1 = time.time()\nprint(end1 - start1)\n#426s\n\n"

In [17]:
"""
start4 = time.time()
samples['S2Region'] = [z.parent(s2level) for z in samples['S2CellID']]
end4 = time.time()
print(end4 - start4)
#67s
"""



"\nstart4 = time.time()\nsamples['S2Region'] = [z.parent(s2level) for z in samples['S2CellID']]\nend4 = time.time()\nprint(end4 - start4)\n#67s\n"

In [18]:
#print(tabulate(samples[0:11000000:1000000], headers='keys', tablefmt='github', showindex=False))

In [19]:
#samples.nunique()